This is a basic Unsupervised structure trainnig.
Place can be reformed: loss func,decoding. not implete val and test yet

# IMPORT

In [2]:
!git clone https://github.com/strivn/idl-project

Cloning into 'idl-project'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 207 (delta 113), reused 94 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (207/207), 2.61 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import os
import sys
import wandb

In [4]:

import wandb
wandb.login(key="752c6c684acf41e944cecdf53f1800860e24801d")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xuanyis (xuanyis-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA A100-SXM4-40GB


In [6]:
sys.path.append("/content/idl-project/src")
from model import load_fo_model, load_ba_model
from data import load_flan_dataset

OUTPUT_DIR = "output/"

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

Using device: cuda


# Load model and dataset

In [8]:
fomodel, fotokenizer = load_fo_model()
bamodel, batokenizer = load_ba_model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fomodel.to(device)
bamodel.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwi

In [9]:
batch_size = 16
dataset = load_flan_dataset().batch(batch_size=batch_size)

README.md:   0%|          | 0.00/6.83k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/2167 [00:00<?, ?it/s]

Dataset loaded successfully


In [10]:
print(fotokenizer.pad_token)
print(fotokenizer.pad_token_id)
print(fotokenizer.eos_token)
print(batokenizer.pad_token)
print(batokenizer.pad_token_id)
print(batokenizer.eos_token)

None
None
<|endoftext|>
None
None
None


In [11]:
print("Fo tokenizer added tokens:")
print(fotokenizer.added_tokens_decoder)

print("\nBa tokenizer added tokens:")
print(batokenizer.added_tokens_decoder)

Fo tokenizer added tokens:
{0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50257: AddedToken("                     ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50258: AddedToken("                    ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50259: AddedToken("                   ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False), 50260: Adde

In [12]:
fotokenizer.add_special_tokens({'pad_token': "<|padding|>"})
fotokenizer.pad_token_id = fotokenizer.convert_tokens_to_ids("<|padding|>")
fomodel.config.pad_token_id = fotokenizer.pad_token_id

batokenizer.add_special_tokens({'pad_token': "<pad>"})
batokenizer.pad_token_id = batokenizer.convert_tokens_to_ids("<pad>")
bamodel.config.pad_token_id = batokenizer.pad_token_id


print(fotokenizer.pad_token, fotokenizer.pad_token_id)
print(fotokenizer.eos_token, fotokenizer.eos_token_id)

<|padding|> 1
<|endoftext|> 0


In [13]:
def preprocess_dual(example, fo_tokenizer, ba_tokenizer):
    fo_combined = f"{example['inputs']}\n{example['targets']}{fo_tokenizer.eos_token}"
    fo_tok = fo_tokenizer(
        fo_combined,
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt"
    )
    fo_input_ids = fo_tok["input_ids"].squeeze(0)
    fo_labels = fo_input_ids.clone()
    fo_labels[:-1] = fo_input_ids[1:]
    fo_labels[fo_input_ids == fo_tokenizer.pad_token_id] = -100
    fo_labels[fo_input_ids == fo_tokenizer.eos_token_id] = -100


    ba_combined = f"{example['targets']}\n{example['inputs']}"
    ba_tok = ba_tokenizer(
        ba_combined,
        truncation=True,
        padding="max_length",
        max_length=768,
        return_tensors="pt"
    )
    ba_input_ids = ba_tok["input_ids"].squeeze(0).flip(dims=[0])
    ba_labels = ba_input_ids.clone()
    ba_labels = torch.roll(ba_labels, shifts=1, dims=0)
    ba_labels[0] = -100

    return {
        "fo_input_ids": fo_input_ids,
        "fo_labels": fo_labels,
        "ba_input_ids": ba_input_ids,
        "ba_labels": ba_labels
    }

In [14]:
examples = next(iter(dataset))

for i in range(3):
    ex = {
        "inputs": examples["inputs"][i],
        "targets": examples["targets"][i]
    }

    dual = preprocess_dual(ex, fotokenizer, batokenizer)

    print(f"===== Example {i+1} =====")

    fo_text = fotokenizer.decode(dual["fo_input_ids"], skip_special_tokens=True)
    print("[FO] Forward decoded:")
    print(fo_text)

    ba_text = batokenizer.decode(dual["ba_input_ids"].flip(dims=[0]), skip_special_tokens=True)
    print("[BA] Backward decoded:")
    print(ba_text)

    print("-" * 60)

===== Example 1 =====
[FO] Forward decoded:
The man be showing his toys to adults and not just kids.. So what could be the question?
Question followed by answer: If "A man wearing dark green shirt and sweatpants is showing off a stuffed toy." does that mean that "A man is showing his stuffed toy to all the kids."?
Options:
- yes
- it is not possible to tell
- no
it is not possible to tell

The infant cannot be crying and asleep at the same time.
The question and answer are below.
Can we conclude from "Two young boys hold a crying infant while both are sitting in the same chair." that "The infant is fast asleep."?
Options:
- yes
- no
- it is not possible to tell
no

At a podium speaking does not necessarily mean speaking out about domestic violence.
The question and answer are below.
Premise: "A woman in black with a tan suit jacket is standing at a podium speaking to an audience."
Hypothesis: "A woman is speaking out about domestic violence to an audience of school kids."
Do we know th

In [15]:
from torch.utils.data import Dataset, DataLoader

class DualDataset(Dataset):
    def __init__(self, hf_dataset, fo_tokenizer, ba_tokenizer, max_samples=None):
        self.fo_tokenizer = fo_tokenizer
        self.ba_tokenizer = ba_tokenizer

        self.data = []
        for i, example in enumerate(hf_dataset):
            if max_samples and i >= max_samples:
                break
            try:
                processed = preprocess_dual(example, fo_tokenizer, ba_tokenizer)
                self.data.append(processed)
            except:
                continue  # skip error

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def get_loader(self, batch_size=8, shuffle=True):
        def collate_fn(batch):
            fo_input_ids = torch.stack([item["fo_input_ids"] for item in batch])
            fo_labels    = torch.stack([item["fo_labels"] for item in batch])
            ba_input_ids = torch.stack([item["ba_input_ids"] for item in batch])
            ba_labels    = torch.stack([item["ba_labels"] for item in batch])
            return {
                "fo_input_ids": fo_input_ids,
                "fo_labels": fo_labels,
                "ba_input_ids": ba_input_ids,
                "ba_labels": ba_labels
            }

        return DataLoader(
            self,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=collate_fn
        )

In [16]:
dual_dataset = DualDataset(dataset, fotokenizer, batokenizer, max_samples=1000)
loader = dual_dataset.get_loader(batch_size=8)

# 测试 batch
batch = next(iter(loader))
print("Fo shape:", batch["fo_input_ids"].shape)
print("Ba shape:", batch["ba_input_ids"].shape)

Fo shape: torch.Size([8, 768])
Ba shape: torch.Size([8, 768])


# Train

In [26]:
import torch
import torch.nn as nn
from torch.optim import AdamW

wandb.init(project="dual-training", name="fo-ba-loop-run")
def train_dual(fomodel, fotokenizer, bamodel, batokenizer, dataloader, device, epochs=3):
    fomodel.train()
    bamodel.train()

    fo_optimizer = AdamW(fomodel.parameters(), lr=5e-5)
    ba_optimizer = AdamW(bamodel.parameters(), lr=5e-5)

    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    global_step = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}")
        for batch in dataloader:
            fo_input_ids = batch['fo_input_ids'].to(device)
            fo_labels = batch['fo_labels'].to(device)
            ba_input_ids = batch['ba_input_ids'].to(device)
            ba_labels = batch['ba_labels'].to(device)

            fo_outputs = fomodel(fo_input_ids, labels=fo_labels)
            ba_outputs = bamodel(ba_input_ids, labels=ba_labels)

            fo_loss = fo_outputs.loss
            ba_loss = ba_outputs.loss
            total_loss = fo_loss + ba_loss

            fo_optimizer.zero_grad()
            ba_optimizer.zero_grad()
            total_loss.backward()
            fo_optimizer.step()
            ba_optimizer.step()

            print(f"Loss: {total_loss.item():.4f}")

            wandb.log({
                "global_step": global_step,
                "fo_loss": fo_loss.item(),
                "ba_loss": ba_loss.item(),
                "total_loss": total_loss.item()
            })
            global_step += 1

        torch.save(fomodel.state_dict(), f"{OUTPUT_DIR}/fomodel_epoch{epoch}.pt")
        torch.save(bamodel.state_dict(), f"{OUTPUT_DIR}/bamodel_epoch{epoch}.pt")
        print(f"✅ Saved models for epoch {epoch}")



In [27]:
train_dual(fomodel, fotokenizer, bamodel, batokenizer, loader, device, epochs=3)


Epoch 1
Loss: 10.6803
Loss: 8.1949
Loss: 6.8684
Loss: 6.4190
Loss: 7.2051
Loss: 6.0421
Loss: 5.5307
Loss: 5.6309
Loss: 5.7051
Loss: 5.0969
Loss: 4.5590
Loss: 4.7691
Loss: 4.4322
Loss: 4.7142
Loss: 5.2578
Loss: 4.4186
Loss: 4.8076
Loss: 3.9309
Loss: 4.3198
Loss: 3.9100
Loss: 4.1914
Loss: 3.7015
Loss: 3.4571
Loss: 5.2027
Loss: 3.6246
Loss: 4.7221
Loss: 4.1604
Loss: 3.3608
Loss: 4.0064
Loss: 3.5127
Loss: 3.9968
Loss: 3.9098
Loss: 3.9907
Loss: 3.7659
Loss: 3.5039
Loss: 3.9129
Loss: 4.2642
Loss: 3.8372
Loss: 3.3934
Loss: 4.0325
Loss: 3.4820
Loss: 3.8461
Loss: 3.0927
Loss: 3.7372
Loss: 3.5633
Loss: 3.4080
Loss: 3.4529
Loss: 3.2345
Loss: 2.9615
Loss: 4.3101
Loss: 3.2781
Loss: 3.9284
Loss: 2.9825
Loss: 3.9861
Loss: 2.5504
Loss: 3.7853
Loss: 3.7833
Loss: 3.7039
Loss: 3.4394
Loss: 2.9555
Loss: 2.8184
Loss: 4.4571
Loss: 3.9264
Loss: 4.2125
Loss: 3.1547
Loss: 3.7105
Loss: 2.7692
Loss: 3.1298
Loss: 3.2759
Loss: 4.4972
Loss: 3.7225
Loss: 3.5080
Loss: 3.5102
Loss: 3.7545
Loss: 3.3482
Loss: 3.6295
Lo